In [1]:
import numpy as np
import pandas as pd
import math
import os
from tqdm import tqdm


In [2]:
vertical_type = "coronal"

In [3]:

DATA_DIR = "/root/autodl-tmp/cervical_spine/"

In [4]:
xray_df = pd.read_csv(os.path.join(DATA_DIR, 'meta_xray.csv')).set_index('UID')
xray_df = xray_df[xray_df.direction == 2]
xray_df.head()

,direction,z_spacing,pixel_spacing,aspect,width,height
UID,,,,,,
1.2.826.0.1.3680043.10001,2,0.625,0.253906,2.461541,512,659
1.2.826.0.1.3680043.10005,2,0.625,0.298828,2.091504,512,541
1.2.826.0.1.3680043.10014,2,0.800,0.234000,3.418803,512,882
1.2.826.0.1.3680043.10016,2,0.313,0.275391,1.136566,512,733
1.2.826.0.1.3680043.10032,2,0.625,0.320313,1.951216,512,626


In [5]:
bbox_df = pd.read_csv(os.path.join(DATA_DIR, 'meta_boundary_throw_anomaly.csv')).set_index('UID')
bbox_df.head()

,coronal_left,coronal_top,coronal_right,coronal_bottom,sagittal_left,sagittal_top,sagittal_right,sagittal_bottom
UID,,,,,,,,
1.2.826.0.1.3680043.10001,74,54,185,213,58,54,164,213
1.2.826.0.1.3680043.10005,66,10,172,142,31,9,166,126
1.2.826.0.1.3680043.10014,79,61,170,239,73,61,182,239
1.2.826.0.1.3680043.10016,83,49,176,192,62,49,166,195
1.2.826.0.1.3680043.10032,74,68,168,223,48,67,179,223


In [6]:
concat_df = pd.concat([xray_df, bbox_df], axis=1)
concat_df.head()

,direction,z_spacing,pixel_spacing,aspect,width,height,coronal_left,coronal_top,coronal_right,coronal_bottom,sagittal_left,sagittal_top,sagittal_right,sagittal_bottom
UID,,,,,,,,,,,,,,
1.2.826.0.1.3680043.10001,2,0.625,0.253906,2.461541,512,659,74,54,185,213,58,54,164,213
1.2.826.0.1.3680043.10005,2,0.625,0.298828,2.091504,512,541,66,10,172,142,31,9,166,126
1.2.826.0.1.3680043.10014,2,0.800,0.234000,3.418803,512,882,79,61,170,239,73,61,182,239
1.2.826.0.1.3680043.10016,2,0.313,0.275391,1.136566,512,733,83,49,176,192,62,49,166,195
1.2.826.0.1.3680043.10032,2,0.625,0.320313,1.951216,512,626,74,68,168,223,48,67,179,223


In [7]:
concat_df.insert(1, 'left', None)
concat_df.insert(1, 'top', None)
concat_df.insert(1, 'right', None)
concat_df.insert(1, 'bottom', None)
concat_df.head()

,direction,bottom,right,top,left,z_spacing,pixel_spacing,aspect,width,height,coronal_left,coronal_top,coronal_right,coronal_bottom,sagittal_left,sagittal_top,sagittal_right,sagittal_bottom
UID,,,,,,,,,,,,,,,,,,
1.2.826.0.1.3680043.10001,2,None,None,None,None,0.625,0.253906,2.461541,512,659,74,54,185,213,58,54,164,213
1.2.826.0.1.3680043.10005,2,None,None,None,None,0.625,0.298828,2.091504,512,541,66,10,172,142,31,9,166,126
1.2.826.0.1.3680043.10014,2,None,None,None,None,0.800,0.234000,3.418803,512,882,79,61,170,239,73,61,182,239
1.2.826.0.1.3680043.10016,2,None,None,None,None,0.313,0.275391,1.136566,512,733,83,49,176,192,62,49,166,195
1.2.826.0.1.3680043.10032,2,None,None,None,None,0.625,0.320313,1.951216,512,626,74,68,168,223,48,67,179,223


In [8]:
def get_original_position(size, bbox, tol=0.1):
    """
    original position out of 512
    :param size: [h, w] usually 512
    :param bbox: numpy 4, bbox position [left, top, right, bottom] max number 256
    :return:
    """
    h, w = size
    left, top, right, bottom = bbox * max(size) / 256
    pad = (max(size) - min(size)) // 2  # pad number of left or top
    if w > h:
        left, top, right, bottom = [left, -pad + top, right, -pad + bottom]
    else:
        left, top, right, bottom = [-pad + left, top, -pad + right, bottom]
    w_tol = (right - left) * tol
    h_tol = (bottom - top) * tol
    left, top, right, bottom = left - w_tol, top - h_tol, right + w_tol, bottom + h_tol
    return int(math.floor(max(0, left))), int(math.floor(max(0, top))), int(math.ceil(min(w, right))), int(math.ceil(min(h, bottom)))

In [9]:

for i in tqdm(range(len(concat_df))):
    row =  concat_df.iloc[i]
    UID = row.name
    series =  concat_df.loc[UID]
    # print(series[["height", "width"]])
    left, top, right, bottom = get_original_position(series[["height", "width"]].values, concat_df.loc[UID, [f"{vertical_type}_left", f"{vertical_type}_top",f"{vertical_type}_right",f"{vertical_type}_bottom"]], tol=0.1)
    # print(np.asarray([[left, top, right, bottom]]).repeat(len( concat_df.loc[UID]), axis=0).transpose().shape)
    #  concat_df.loc[UID, ['left','top','right','bottom']] = np.asarray([[left, top, right, bottom]]).repeat(len( concat_df.loc[UID]), axis=0)
    concat_df.loc[UID, ['left', 'top', 'right', 'bottom']] = left, top, right, bottom
concat_df.head()

100%|██████████| 2011/2011 [00:05<00:00, 373.38it/s]


,direction,bottom,right,top,left,z_spacing,pixel_spacing,aspect,width,height,coronal_left,coronal_top,coronal_right,coronal_bottom,sagittal_left,sagittal_top,sagittal_right,sagittal_bottom
UID,,,,,,,,,,,,,,,,,,
1.2.826.0.1.3680043.10001,2,590,432,98,88,0.625,0.253906,2.461541,512,659,74,54,185,213,58,54,164,213
1.2.826.0.1.3680043.10005,2,328,372,0,103,0.625,0.298828,2.091504,512,541,66,10,172,142,31,9,166,126
1.2.826.0.1.3680043.10014,2,882,433,148,55,0.800,0.234000,3.418803,512,882,79,61,170,239,73,61,182,239
1.2.826.0.1.3680043.10016,2,591,421,99,101,0.313,0.275391,1.136566,512,733,83,49,176,192,62,49,166,195
1.2.826.0.1.3680043.10032,2,584,377,128,100,0.625,0.320313,1.951216,512,626,74,68,168,223,48,67,179,223


In [10]:
concat_df.insert(1, f'{vertical_type}_index', None)

In [11]:
df = pd.DataFrame()

In [12]:
for i in tqdm(range(len(concat_df))):
    s = concat_df.iloc[i]
    UID = s.name
    if vertical_type == 'sagittal':
        left, top, right, bottom = get_original_position(s[["height", "width"]].values, concat_df.loc[UID, [f"coronal_left", f"coronal_top",f"coronal_right",f"coronal_bottom"]], tol=0.1)
        right = min(511, right)
        new_df = concat_df.loc[[UID] * (right - left + 1)]
        new_df[f"{vertical_type}_index"] = np.arange(left, right + 1)
        df = pd.concat([df, new_df])
    else:
        left, top, right, bottom = get_original_position(s[["height", "width"]].values, concat_df.loc[UID, [f"sagittal_left", f"sagittal_top",f"sagittal_right",f"sagittal_bottom"]], tol=0.1)
        right = min(511, right)
        new_df = concat_df.loc[[UID] * (right - left + 1)]
        new_df[f"{vertical_type}_index"] = np.arange(left, right + 1)
        df = pd.concat([df, new_df])

df.head()

100%|██████████| 2011/2011 [01:10<00:00, 28.53it/s]


,direction,coronal_index,bottom,right,top,left,z_spacing,pixel_spacing,aspect,width,height,coronal_left,coronal_top,coronal_right,coronal_bottom,sagittal_left,sagittal_top,sagittal_right,sagittal_bottom
UID,,,,,,,,,,,,,,,,,,,
1.2.826.0.1.3680043.10001,2,49,590,432,98,88,0.625,0.253906,2.461541,512,659,74,54,185,213,58,54,164,213
1.2.826.0.1.3680043.10001,2,50,590,432,98,88,0.625,0.253906,2.461541,512,659,74,54,185,213,58,54,164,213
1.2.826.0.1.3680043.10001,2,51,590,432,98,88,0.625,0.253906,2.461541,512,659,74,54,185,213,58,54,164,213
1.2.826.0.1.3680043.10001,2,52,590,432,98,88,0.625,0.253906,2.461541,512,659,74,54,185,213,58,54,164,213
1.2.826.0.1.3680043.10001,2,53,590,432,98,88,0.625,0.253906,2.461541,512,659,74,54,185,213,58,54,164,213


In [13]:
len(df)

555194

In [14]:
df.loc["1.2.826.0.1.3680043.10001"]

,direction,coronal_index,bottom,right,top,left,z_spacing,pixel_spacing,aspect,width,height,coronal_left,coronal_top,coronal_right,coronal_bottom,sagittal_left,sagittal_top,sagittal_right,sagittal_bottom
UID,,,,,,,,,,,,,,,,,,,
1.2.826.0.1.3680043.10001,2,49,590,432,98,88,0.625,0.253906,2.461541,512,659,74,54,185,213,58,54,164,213
1.2.826.0.1.3680043.10001,2,50,590,432,98,88,0.625,0.253906,2.461541,512,659,74,54,185,213,58,54,164,213
1.2.826.0.1.3680043.10001,2,51,590,432,98,88,0.625,0.253906,2.461541,512,659,74,54,185,213,58,54,164,213
1.2.826.0.1.3680043.10001,2,52,590,432,98,88,0.625,0.253906,2.461541,512,659,74,54,185,213,58,54,164,213
1.2.826.0.1.3680043.10001,2,53,590,432,98,88,0.625,0.253906,2.461541,512,659,74,54,185,213,58,54,164,213
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1.2.826.0.1.3680043.10001,2,373,590,432,98,88,0.625,0.253906,2.461541,512,659,74,54,185,213,58,54,164,213
1.2.826.0.1.3680043.10001,2,374,590,432,98,88,0.625,0.253906,2.461541,512,659,74,54,185,213,58,54,164,213
1.2.826.0.1.3680043.10001,2,375,590,432,98,88,0.625,0.253906,2.461541,512,659,74,54,185,213,58,54,164,213


In [15]:
df.to_csv(f'infer_{vertical_type}_position.csv')

In [16]:
df[df.coronal_index == 512]

,direction,coronal_index,bottom,right,top,left,z_spacing,pixel_spacing,aspect,width,height,coronal_left,coronal_top,coronal_right,coronal_bottom,sagittal_left,sagittal_top,sagittal_right,sagittal_bottom
UID,,,,,,,,,,,,,,,,,,,


In [18]:
df.bottom.min()

215

In [ ]:
df.right.min()